<a href="https://colab.research.google.com/github/akshaypt7/amazon_sales_prediction/blob/main/my_project_01_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 7.6 MB/s 
     |████████████████████████████████| 188 kB 70.0 MB/s 
     |████████████████████████████████| 1.2 MB 51.8 MB/s 
     |████████████████████████████████| 46 kB 6.1 MB/s 
     |████████████████████████████████| 53 kB 3.2 MB/s 
     |████████████████████████████████| 51 kB 238 kB/s 
Mounted at /content/gdrive


In [8]:
#hide
from fastbook import *
# from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
# from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [9]:
from sklearn.model_selection import ParameterGrid
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers import Dense, Flatten, Dropout, Input, Embedding, Reshape
from keras.layers import concatenate
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import pickle
import csv
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

from sklearn.model_selection import train_test_split

import pandas as pd

### Improvements from 1.01

Here we are planning to improve the results we got from 1.2, by introducing Embedding. We create embeddings for the neural network we will also try to use the same in random forest and see its performance.

We need to refresh our knowledge on Embeddgings. first we will write about it and then we can also look at the excersice we have done earlier.

What do know about embeddings?

We know that it looks like a list of number, and we can define how many number should represent a quantity. that can then stand of some measurement of the quantity we are trying a replicate.

Technically speaking we can say that embedding is a computational shortcut for one-hot encoding, yes and we can say that even the list of numbers which we talked about earlier also be thought in those terms. 

we can also create embeddings, and i think where it differs with one hot encoding is that it will have continous values not just zero and one. 

We can even think of reading that embedding paper again.

One thing which I cleared right now is there is two concepts embedding and entity embedding.

Entity Embedding is where we define a quantity by a list of numbers and embedding is the computational shortcut.

**Refer this - https://towardsdatascience.com/enhancing-categorical-features-with-entity-embeddings-e6850a5e34ff**

In [13]:
infile = open('/content/gdrive/MyDrive/bluebook/df_main.pkl','rb')
df_main = pickle.load(infile)

In [ ]:
df_main.head()

,date,Asin,Item Promo Discount,price,hour,Quantity,Invoice Amount
0,2021-01-04,B07KYFHTGF,0.00,1995.00,12.0,1.0,1995.00
1,2021-01-08,B07TM3LRVB,-56.75,1078.25,20.0,1.0,1078.25
2,2021-01-09,B08976V1BZ,-79.95,1519.05,22.0,1.0,1519.05
3,2021-01-11,B07TM3LRVB,0.00,1135.00,10.0,1.0,1135.00
4,2021-01-12,B08332221J,0.00,1499.00,12.0,1.0,1499.00


In [21]:
df_main = add_datepart(df_main,'date')

In [22]:
df_main.columns

Index(['Asin', 'Item Promo Discount', 'price', 'hour', 'Quantity',
       'Invoice Amount', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed'],
      dtype='object')

In [16]:
import math

In [23]:
# embedding for ASiN
n_unique_asins = df_main.Asin.nunique()
n_dim_asins = int(math.sqrt(n_unique_asins))


input_asin = Input(shape=(1,))
output_asin = Embedding(input_dim = n_unique_asins, output_dim= n_dim_asins, name='asin')(input_asin)
output_asin = Reshape(target_shape=(n_dim_asins,))(output_asin)

In [24]:
# embedding for hour
n_unique_hour = df_main.hour.nunique()
n_dim_hours = int(math.sqrt(n_unique_hour))

input_hour = Input(shape=(1,))
output_hour = Embedding(input_dim= n_unique_hour, output_dim=n_dim_hours,name='hours')(input_hour)
output_hour = Reshape( target_shape= (n_dim_hours,))(output_hour)

In [25]:
# day_of week
n_unique_day_of_week = df_main['Dayofweek'].nunique()
n_dim_dayofweek = 3

input_dayofweek = Input(shape=(1,))
output_dayofweek = Embedding(input_dim= n_unique_day_of_week, output_dim=n_dim_dayofweek,name='dayofweek')(input_dayofweek)
output_dayofweek = Reshape( target_shape= (n_dim_dayofweek,))(output_dayofweek)

In [26]:
# Month
n_unique_month = df_main['Month'].nunique()
n_dim_month = 7

input_month = Input(shape=(1,))
output_month = Embedding(input_dim= n_unique_month, output_dim=n_dim_month,name='month')(input_month)
output_month = Reshape( target_shape= (n_dim_month,))(output_month)

In [27]:
# Day (1-31)
n_unique_day = df_main['Day'].nunique()
n_dim_day = int(math.sqrt(n_unique_day))

input_day = Input(shape=(1,))
output_day= Embedding(input_dim= n_unique_day, output_dim=n_dim_day,name='day')(input_day)
output_day = Reshape( target_shape= (n_dim_day,))(output_day)

In [28]:
# Week

n_unique_week = df_main['Week'].nunique()
n_dim_week = 30

input_week = Input(shape=(1,))
output_week= Embedding(input_dim= n_unique_week, output_dim=n_dim_week,name='week')(input_week)
output_week = Reshape( target_shape= (n_dim_week,))(output_week)


In [30]:
input_model = [input_asin, input_hour, input_dayofweek, input_month, input_day, input_week ]
output_embeddings = [output_asin, output_hour, output_dayofweek, output_month, output_day, output_week ] 
output_model = Concatenate()(output_embeddings)

In [31]:

output_model = Dense(1000, kernel_initializer='uniform')(output_model)
output_model = Activation('relu')(output_model)

output_model = Dense(500, kernel_initializer='uniform')(output_model)
output_model = Activation('relu')(output_model)

output_model = Dense(1)(output_model)
output_model = Activation('sigmoid')(output_model)

In [32]:
#Define the embedding NN model
#your code here

modelNN_emb = KerasModel(inputs = input_model, outputs = output_model)
modelNN_emb.compile(optimizer='adam',loss='mean_absolute_error')

In [ ]:
input_model

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_2')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_3')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_4')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_5')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_6')>]

In [ ]:
output_model

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'activation_2')>

In [ ]:
df_main.columns

Index(['Asin', 'Item Promo Discount', 'price', 'hour', 'Quantity',
       'Invoice Amount', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed'],
      dtype='object')

In [ ]:
input_asin, input_hour, input_dayofweek, input_month, input_day, input_week ]

In [ ]:
def split_features(X):
    X_list = []

    asin = X[:, [0]]
    X_list.append(asin)

    hour = X[:, [3]]
    X_list.append(hour)

    dayofweek = X[:, [10]]#
    X_list.append(dayofweek)

    month = X[:, [7]]
    X_list.append(month)

    day = X[:, [9]]
    X_list.append(day)

    week = X[:, [8]]
    X_list.append(week)

    return X_list

In [ ]:
def preprocessing(X):
    X_list = split_features(X)
    return X_list

In [ ]:
procs = [Categorify, FillMissing]

In [ ]:

dep_var = 'Quantity'

In [ ]:
cont, cat = cont_cat_split( df_main,1,dep_var= dep_var)

In [ ]:
cont =['Item Promo Discount',
 'price',
 'Invoice Amount',
 'Year',
 'Elapsed']

In [ ]:
cat = ['Asin',
 'Is_month_end',
 'Is_month_start',
 'Is_quarter_end',
 'Is_quarter_start',
 'Is_year_end',
 'Is_year_start',
 'hour','Month','Week', 'Day','Dayofweek','Dayofyear',]

In [ ]:
# SPLITS 
condition = (df_main.Year < 2020) | (df_main.Month < 8)
train_idx = np.where(condition)[0]
valid_idx = np.where(~condition)[0]

In [ ]:
len(train_idx), len(valid_idx)

(7517, 3429)

In [ ]:
splits = (list(train_idx),list(valid_idx))

In [ ]:

to = TabularPandas(df_main, procs= procs,cat_names= cat, cont_names=cont, splits= splits,y_names=dep_var)

In [ ]:
x_train,y_train = to.train.xs, to.train.y
x_valid, y_valid = to.valid.xs, to.valid.y

In [ ]:
df_main.columns

Index(['Asin', 'Item Promo Discount', 'price', 'hour', 'Quantity',
       'Invoice Amount', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed'],
      dtype='object')

In [ ]:
modelNN_emb.checkpointer = ModelCheckpoint(filepath="/content/drive/MyDrive/bluebook/best_model_weights.hdf5", verbose=1, save_best_only=True)
modelNN_emb.fit(preprocessing(x_train.values), y_train, epochs = 10, batch_size = 128, validation_data= (preprocessing(x_valid.values),y_valid), callbacks=[modelNN_emb.checkpointer])


Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
df_main.columns

Index(['Asin', 'Item Promo Discount', 'price', 'hour', 'Quantity',
       'Invoice Amount', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed'],
      dtype='object')

### Fastai Approach

In [ ]:
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
dls = to.dataloaders(1024)

In [ ]:
learn = tabular_learner(dls, layers=[500,250], n_out=1)
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,time
0,1.742647,3.595780,00:00
1,1.612347,50.271015,00:00
2,1.511296,260.635101,00:00
3,1.424649,215.328995,00:00
4,1.357270,118.722649,00:00


In [ ]:
def embed_features(learner, xs):
  """
  learner: fastai Learner used to train the neural net
  xs: DataFrame containing input variables. Categorical values are defined by their rank. 
 ::return:: copy of `xs` with embeddings replacing each categorical variable
  """
  xs = xs.copy()
  for i,col in enumerate(learn.dls.cat_names):
    
    # get matrix containing each row's embedding vector
    emb = learn.model.embeds[i]
    emb_data = emb(tensor(xs[col], dtype=torch.int64))
    emb_names = [f'{col}_{j}' for j in range(emb_data.shape[1])]
    
    # join the embedded category and drop the old feature column
    feat_df = pd.DataFrame(data=emb_data, index=xs.index,               
                           columns=emb_names)
    xs = xs.drop(col, axis=1)
    xs = xs.join(feat_df)
  return xs

In [ ]:
emb_xs = embed_features(learn, to.train.xs)
emb_valid_xs = embed_features(learn, to.valid.xs)

RuntimeError: ignored

### New

In [ ]:
# embedding for ASiN
n_unique_asins = df_main.Asin.nunique()
n_dim_asins = int(math.sqrt(n_unique_asins))


input_asin = Input(shape=(1,))
output_asin = Embedding(input_dim = n_unique_asins, output_dim= n_dim_asins, name='asin')(input_asin)
output_asin = Reshape(target_shape=(n_dim_asins,))(output_asin)

In [ ]:
# embedding for hour
n_unique_hour = df_main.hour.nunique()
n_dim_hours = int(math.sqrt(n_unique_hour))

input_hour = Input(shape=(1,))
output_hour = Embedding(input_dim= n_unique_hour, output_dim=n_dim_hours,name='hours')(input_hour)
output_hour = Reshape( target_shape= (n_dim_hours,))(output_hour)

In [ ]:
# day_of week
n_unique_day_of_week = df_main['Dayofweek'].nunique()
n_dim_dayofweek = 3

input_dayofweek = Input(shape=(1,))
output_dayofweek = Embedding(input_dim= n_unique_day_of_week, output_dim=n_dim_dayofweek,name='dayofweek')(input_dayofweek)
output_dayofweek = Reshape( target_shape= (n_dim_dayofweek,))(output_dayofweek)

In [ ]:
# Month
n_unique_month = df_main['Month'].nunique()
n_dim_month = 7

input_month = Input(shape=(1,))
output_month = Embedding(input_dim= n_unique_month, output_dim=n_dim_month,name='month')(input_month)
output_month = Reshape( target_shape= (n_dim_month,))(output_month)

In [ ]:
# Day (1-31)
n_unique_day = df_main['Day'].nunique()
n_dim_day = int(math.sqrt(n_unique_day))

input_day = Input(shape=(1,))
output_day= Embedding(input_dim= n_unique_day, output_dim=n_dim_day,name='day')(input_day)
output_day = Reshape( target_shape= (n_dim_day,))(output_day)

In [ ]:
# Week

n_unique_week = df_main['Week'].nunique()
n_dim_week = 30

input_week = Input(shape=(1,))
output_week= Embedding(input_dim= n_unique_week, output_dim=n_dim_week,name='week')(input_week)
output_week = Reshape( target_shape= (n_dim_week,))(output_week)


In [ ]:
input_layers = [input_asin, input_hour, input_dayofweek, input_month, input_day, input_week ]
output_layers = [output_asin, output_hour, output_dayofweek, output_month, output_day, output_week ] 

In [ ]:
model = Concatenate()(output_layers)

In [ ]:
# Add a few hidden layers
model = Dense(200, kernel_initializer="uniform")(model)
model = Activation('relu')(model)

model = Dense(100, kernel_initializer="uniform")(model)
model = Activation('relu')(model)
# And finally our output layer
model = Dense(1)(model)
model = Activation('sigmoid')(model)

In [ ]:
model = KerasModel(inputs=input_layers, outputs=model)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 1)]          0                                            
____________________________________________________________________________________________

In [ ]:
opt = SGD(  ,lr=0.05)
model.compile(loss='mse', optimizer=opt, metrics=['mse'])

TypeError: ignored